In [62]:
import os

In [63]:
from bs4 import BeautifulSoup
import cloudscraper
import geopandas as gpd
import pandas as pd

In [64]:
from shapely.geometry import Point
from tqdm.notebook import tqdm

## Get State Data

In [45]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
states_df = gpd.read_file(file_path)

In [46]:
states_df = states_df[["STUSPS", "NAME", "geometry"]]

## Get Population data

In [47]:
state_populations = pd.read_excel(
    "data/NST-EST2023-POP.xlsx", sheet_name=None, engine="openpyxl"
)

In [48]:
state_populations_df = state_populations["NST-EST2023-POP"][
    [
        "table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)",
        "Unnamed: 5",
    ]
]
state_populations_df = state_populations_df.rename(
    columns={
        "table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)": "NAME",
        "Unnamed: 5": "POPULATION",
    }
)
state_populations_df["NAME"] = state_populations_df["NAME"].str[1:]

In [49]:
states_with_population_df = states_df.merge(state_populations_df, on="NAME", how="left")
states_with_population_df = states_with_population_df[
    ["STUSPS", "NAME", "POPULATION", "geometry"]
]

## Get Dollar General Data

In [10]:
scraper = cloudscraper.create_scraper()

In [ ]:
store_list = []
url_base = "https://www.dollargeneral.com/store-directory/"
for i in tqdm(range(len(states_df)), desc="Parsing States"):
    # Get State code
    state_store_list = []
    state_code = states_df.iloc[i]["STUSPS"]

    # If File exists continue
    if os.path.isfile(f"data/states/{state_code}.gpkg"):
        continue

    dollar_general_count_dict[state_code] = 0

    # Get all locations in a state
    url = os.path.join(url_base, state_code)
    r = scraper.get(url)

    if r.status_code != 200:
        print(states_df.iloc[i]["NAME"])
        continue

    soup = BeautifulSoup(r.text, "html.parser")
    locations_ps = soup.find_all("p", {"class": "location-list-item"})

    # Parse all locations in a state
    for location_p in tqdm(locations_ps, desc=f"Parsing Locations In {state_code}"):
        href = location_p.find("a").attrs["href"]
        location_paramters = "/".join(location_p.find("a").attrs["href"].split("/")[5:])
        location_url = os.path.join(url_base, location_paramters)
        r_location = scraper.get(location_url)
        if r_location.status_code != 200:
            print(location_paramters)
            continue
        location_soup = BeautifulSoup(r_location.text, "html.parser")
        location_cards = location_soup.find_all("div", {"class": "store__card"})

        # Multiple stores in locations, parse the stores per location
        for location_card in location_cards:
            try:
                store_id = (
                    location_card.find("a").attrs["href"].split("/")[-1].split(".")[0]
                )
            except Exception as e:
                print(store_id)
                continue

            # Get store lat,long
            store_url = f"https://www.dollargeneral.com/bin/omni/pickup/storeDetails?storeNumber={store_id}"
            r_store = scraper.get(store_url)
            if r_store.status_code != 200:
                print(store_id)
                continue

            # Output to dict then append to list
            store_json = r_store.json()["storeDetails"]
            try:
                store_dict = {
                    "STORE_ID": str(store_id),
                    "STATE": store_json["st"],
                    "geometry": Point(store_json["lo"], store_json["la"]),
                }
            except Exception as e:
                print(store_json, store_id)
                continue
            state_store_list.append(store_dict)
            store_list.append(store_dict)

    state_dollar_general_gdf = gpd.GeoDataFrame(state_store_list, crs=4326)
    state_dollar_general_gdf.to_file(f"data/states/{state_code}.gpkg")

In [ ]:
dollar_general_gdf = gpd.GeoDataFrame(store_list, crs=4326)

#### If the above cells breaks just keep running until complete. Then run the line below

In [29]:
dollar_general_gdfs = []
states_path = "data/states"
for file in os.listdir("data/states"):
    constructed_path = os.path.join(states_path, file)
    if constructed_path.endswith(".gpkg"):
        dollar_general_gdf = gpd.read_file(constructed_path)
        dollar_general_gdfs.append(dollar_general_gdf)

In [30]:
dollar_general_gdf = gpd.GeoDataFrame(pd.concat(dollar_general_gdfs, ignore_index=True))

In [34]:
dollar_general_gdf = dollar_general_gdf.to_crs(9311)
dollar_general_gdf.to_file(f"data/stores.gpkg")

## Combine With States

In [39]:
dollar_general_state_counts_series = dollar_general_gdf.groupby("STATE").size()
dollar_general_state_counts_df = pd.DataFrame(
    {
        "STUSPS": dollar_general_state_counts_series.index,
        "dg": dollar_general_state_counts_series.values,
    }
)

In [54]:
dollar_general_by_states_gdf = states_with_population_df.merge(
    dollar_general_state_counts_df, on="STUSPS", how="left"
)

In [55]:
dollar_general_by_states_gdf = dollar_general_by_states_gdf.fillna(0)
dollar_general_by_states_gdf["dg"] = dollar_general_by_states_gdf["dg"].astype(int)

In [57]:
dollar_general_by_states_gdf["per_1000"] = dollar_general_by_states_gdf["dg"] / (
    dollar_general_by_states_gdf["POPULATION"] / 1000
)
dollar_general_by_states_gdf["per_10k"] = dollar_general_by_states_gdf["dg"] / (
    dollar_general_by_states_gdf["POPULATION"] / 10_000
)
dollar_general_by_states_gdf["per_100k"] = dollar_general_by_states_gdf["dg"] / (
    dollar_general_by_states_gdf["POPULATION"] / 100000
)
dollar_general_by_states_gdf["per_500k"] = dollar_general_by_states_gdf["dg"] / (
    dollar_general_by_states_gdf["POPULATION"] / 500_000
)
dollar_general_by_states_gdf["per_1m"] = dollar_general_by_states_gdf["dg"] / (
    dollar_general_by_states_gdf["POPULATION"] / 1_000_000
)

In [60]:
dollar_general_by_states_gdf = dollar_general_by_states_gdf.dropna()

In [61]:
dollar_general_by_states_gdf = dollar_general_by_states_gdf.to_crs(9311)
dollar_general_by_states_gdf.to_file(f"data/stores_by_states.gpkg")